In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import json
import time
import math
import random
import pickle

import data_module as dm

from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
train_dl, test_dl, train_len, test_len = dm.get_pred_fin()

ValueError: could not determine the shape of object type 'Series'

In [3]:
with open("kmeans_model_fin.pkl", "rb") as f:
    kmeans_fin = pickle.load(f)
    
with open("kmeans_model.pkl", "rb") as f:
    kmeans_beg = pickle.load(f)

In [5]:
it = iter(train_dl)
d = next(it)
poly = d[3]
fin = d[2]
start = d[1]


fin_coord = torch.Tensor([kmeans_fin.cluster_centers_[int(i.item())] for i in fin[:,0]])
fin_coord.shape
print(start[0])
print(fin[0])
print(start.shape)
print(fin.shape)

tensor([1264.])
tensor([159.])
torch.Size([32, 1])
torch.Size([32, 1])


In [26]:
class FinPred(nn.Module):
    def __init__(self):
        super(FinPred, self).__init__()
        
        self.fc0 = nn.Linear(64, 1)
        self.gru = nn.GRU(1, 64, num_layers=2)
        
        self.fc1 = nn.Linear(594+2, 512)
        self.fc2 = nn.Linear(512, 1500)
        self.sm = nn.Softmax(1)
        
    def forward(self, in_feats, start, poly, hidden):
        
        x = start.view((1, -1, 1))
        
        out, hidden = self.gru(x, hidden) # out should be finish cluster
        
        in_feats = torch.cat((in_feats.to(device), out.transpose(0,1), start.transpose(0,1)), dim=1)
        x = F.relu(self.fc1(in_feats))
        x = F.relu(self.fc2(x))
        x = self.sm(x)
        
        return x
    
    def init_hidden(self, bs):
        return torch.zeros((2, bs, 64))

In [27]:
class EquiLoss(nn.Module):
    def __ini__(self):
        super(EquiLoss, self).__init__()
        
    def forward(self, input, target):
        ilon = input[:,0].to(device)
        ilat = input[:,1].to(device)
        
        tlon = target[:,0].to(device)
        tlat = target[:,1].to(device)
        return (6371 * torch.sqrt(((ilon - tlon) * torch.cos((ilat - tlat) / 2))**2 + (ilat - tlat)**2)).mean()

In [28]:
# Trainer helper functions from 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s' % (asMinutes(s))

In [29]:
def train(model, criterion, optimizer, in_feats, start, fin, poly):
    optimizer.zero_grad()
    
    in_feats = in_feats.to(device)
    start = start.to(device)
    fin = fin.to(device)
    poly = poly.to(device)
    
    loss = 0

    hidden = model.init_hidden(32).to(device)
    out = model(in_feats, start, poly, hidden)

    loss += criterion(out, torch.Tensor([kmeans_fin.cluster_centers_[int(i.item())] for i in fin[:,0]]))
    loss.backward()
        
    optimizer.step()
    
    return loss.item()

def trainEpochs(model, n_epochs, print_every=100, eval_every = 5, learning_rate=0.003):
    start_time = time.time()
    print_loss_total = 0  # Reset every print_every

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = EquiLoss()
    ps = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 0.25, 3)

        
    for epoch in range(n_epochs):
        for i, data in enumerate(train_dl):

            in_feats = data[0] # back to data

            if (in_feats.shape[0] != 32):
                continue

            start = data[1] # back to data
            fin = data[2] # back to data
            poly = data[3]
            
            loss = train(model, criterion, optimizer, in_feats, start, fin, poly)
            print_loss_total += loss


            if (i % print_every == 0) and (i != 0): # Change back to i
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('Epoch: %d Elapsed: %s Percent of epoch Complete: (%d%%) %.4f' % (epoch, timeSince(start_time, i / (train_len / 32)),
                                                                  i / (train_len / 32) * 100, print_loss_avg))


            if (i % eval_every == 0) and (i != 0):
                print('*****EVALUATING*****')
                eval_loss = eval_epoch(encoder, decoder, pred, epoch)
                ps.step(eval_loss)





In [30]:
model = FinPred().to(device)

trainEpochs(model, 1, 100, 10000)

AttributeError: 'tuple' object has no attribute 'to'